In [ ]:
#name = twitter_sentiment_tacosushi
#Import the necessary methods from tweepy library as well as json and time
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
import json
import time
import pandas as pd


#We authenticate ourselves as having a twitter app
#Variables that contains the user credentials to access Twitter API 
access_token = "3ads;fkajsdfpoaisdjf;alksdjf"
access_secret = "2uasdl;fajsd;flkjasd;flkajsdf;adfasdfEg"
consumer_key = "xasdf98uoif;wqjer;kandsf"
consumer_secret = "1asd9fp8uijl;qkwef;alksd;iX"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)




#We begin searching our query
#Put your search term
searchquery = "angry"

users =tweepy.Cursor(api.search,q=searchquery).items()
count = 0
start = 0
errorCount=0

#We will be storing our data in file called: happy.json
#file = open('test.json', 'wb') 

#here we tell the program how fast to search 
waitquery = 100      #this is the number of searches it will do before resting
waittime = 2.0          # this is the length of time we tell our program to rest
total_number = 15000     #this is the total number of queries we want
justincase = 1         #this is the number of minutes to wait just in case twitter throttles us



text = [0] * total_number
secondcount = 0
idvalues = [1] * total_number
 #1 is happy; 2 is sad; 3 is angry; 4 is fearful
#Below is where the magic happens and the queries are being made according to our desires above
while secondcount < total_number:
    try:
        user = next(users)
        count += 1
        
        #We say that after every 100 searches wait 5 seconds
        if (count%waitquery == 0):
           time.sleep(waittime)
            #break

    except tweepy.TweepError:
        #catches TweepError when rate limiting occurs, sleeps, then restarts.
        #nominally 15 minnutes, make a bit longer to avoid attention.
        print "sleeping...."
        time.sleep(60*justincase)
        user = next(users)
        
        
    except StopIteration:
        break
    try:
        #print "Writing to JSON tweet number:"+str(count)
        text_value = user._json['text']
        language = user._json['lang']
        #print(text_value)
        print(language)
        
        if "RT" not in text_value:
            if language == "en":
                text[secondcount] = text_value
                secondcount = secondcount + 1
                print("current saved is:")
                print(secondcount)

    except UnicodeEncodeError:
        errorCount += 1
        print "UnicodeEncodeError,errorCount ="+str(errorCount)


print("Creating dataframe:")

d = {"text": text, "id": idvalues}
df = pd.DataFrame(data = d)

df.to_csv('upset.csv', header=True, index=False, encoding='utf-8')

print "completed"


In [ ]:
import pandas
import string

#import my csv file
df = pd.read_csv('unsmile.csv')

#Remove any rows with a "nan" in them
df = df.dropna(axis=0, how = 'any')

#Make it so that any non readable text gets converted into nothing
def removetext(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

#Here I am doing the actual removing
df['text'] = df['text'].apply(removetext)

#Make all my texts lower case
df['text'] = df['text'].apply(lambda x: x.lower())

#Get rid of all weird punctuation and extra lines
df['text'] = df['text'].apply(lambda x: x.replace('.',' '))
df['text'] = df['text'].apply(lambda x: x.replace('\n',' '))
df['text'] = df['text'].apply(lambda x: x.replace('?',' '))
df['text'] = df['text'].apply(lambda x: x.replace('!',' '))
df['text'] = df['text'].apply(lambda x: x.replace('"',' '))
df['text'] = df['text'].apply(lambda x: x.replace(';',' '))
df['text'] = df['text'].apply(lambda x: x.replace('#',' '))
df['text'] = df['text'].apply(lambda x: x.replace('&amp',' '))
df['text'] = df['text'].apply(lambda x: x.replace(',',' '))

#Here I get each unique keyword from my dataframe
array = df['text'].str.split(' ', expand=True).stack().value_counts()
#print(array) to see what this looks like

#I make a dataframe of the words and the frequency with which the words appear 
d = {'word': array.index, 'frequency':array}
df2 = pd.DataFrame(data = d)

#I get rid of any words that are mentioned less than 10 times
df2['frequency'] = df2['frequency'][df2['frequency'] > 10] 

#Remove any rows with a "nan" in them
df2 = df2.dropna(axis=0, how = 'any')

#Drop any obvious signs of these words being :(
df2 = df2.drop([':(','https://t',':((', ':(((', ':((((', ':(((((', ':', '(', ''])

#Convert my dataframe into a csv file
df2.to_csv('unsmile_words.csv', header=True, index=False, encoding='utf-8')


In [ ]:
import pandas as pd

#import my csv files
happy = pd.read_csv('happy_words.csv')
sad = pd.read_csv('sad_words.csv')
unsmile = pd.read_csv('unsmile_words.csv')
fun = pd.read_csv('fun_words.csv')


wordbag = pd.concat([happy,sad,unsmile,fun]).drop_duplicates(subset = 'word').reset_index(drop=True)

print(wordbag)

wordbag.to_csv('wordbag.csv')


In [ ]:
import pandas as pd




df = pd.read_csv('unsmile.csv')

#Remove any rows with a "nan" in them
df = df.dropna(axis=0, how = 'any')

#Make it so that any non readable text gets converted into nothing
def removetext(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

#Here I am doing the actual removing
df['text'] = df['text'].apply(removetext)

#Make all my texts lower case
df['text'] = df['text'].apply(lambda x: x.lower())

#Get rid of all weird punctuation and extra lines
df['text'] = df['text'].apply(lambda x: x.replace('.',' '))
df['text'] = df['text'].apply(lambda x: x.replace('\n',' '))
df['text'] = df['text'].apply(lambda x: x.replace('?',' '))
df['text'] = df['text'].apply(lambda x: x.replace('!',' '))
df['text'] = df['text'].apply(lambda x: x.replace('"',' '))
df['text'] = df['text'].apply(lambda x: x.replace(';',' '))
df['text'] = df['text'].apply(lambda x: x.replace('#',' '))
df['text'] = df['text'].apply(lambda x: x.replace(',',' '))

#split all the words in my text
df['text']= df['text'].str.split()

df.to_csv('unsmile_split.csv')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#Read the files with the tweets
fun = pd.read_csv('fun_split.csv')
happy = pd.read_csv('happy_split.csv')
unsmile = pd.read_csv('unsmile_split.csv')
sad = pd.read_csv('sad_split.csv')

#Get the wordbag
wordbag = pd.read_csv('wordbag.csv')
wordbag = wordbag.drop_duplicates()

#Classify tweets that had the word fun or happy in them as 1 (positive)
#and the others as 0 (negative)
fun['type'] = 1
happy['type'] = 1
unsmile['type'] = 0
sad['type'] = 0
   
#Join all of the dataframes into one big one for easier manipulation of a test/train split
df = pd.concat([happy,sad,unsmile,fun]).reset_index(drop=True)

#Create a test and train set by using the sklearn function train_test_split
train, test = train_test_split(df, test_size=0.2)

#Seperate the train data into a positive and negative set
train_positive = train[train['type'] ==1]
train_negative = train[train['type'] ==0]

positive_instance = len(train_positive)
negative_instance = len(train_negative)
print(positive_instance)
print(negative_instance)

#Create your frequency table
frequency['word'] = wordbag['word']

word_bank = [0]*len(frequency)
positive = [0]*len(frequency)
negative = [0]*len(frequency)

#Go over all the words in the frequency table
for i in range(len(frequency)):
    
    #Get the word in the frequency table at a given row
    word = frequency['word'].iloc[i]
    word_bank[i] = word
             
    #Convert the word and attached single colons ont oboth sides of the word
    check = str("'") + word + str("'")
    
    #Count the number of instances that have the word at least once
    count = 0  
    
    #this iterates through each of the tweets in the positive train set
    for j in range(len(train_positive)):
        #This checks to see the number of time the said word appears in a given tweet
        appears = train_positive['text'].iloc[j].count(check)
        
        #If the word appears at least once, we count it as that tweet having it
        #We sum over all the tweets that the word appears at least once in 
        if appears > 0:
            count = count + 1
    positive[i] = count
            
    #Does the same thing but for negative numbers
    count = 0  
    for k in range(len(train_negative)):
        appears = train_negative['text'].iloc[k].count(check)
        if appears > 0:
            count = count + 1
    negative[i] = count
    print(i)

d = {'word': word_bank, 'positive': positive, 'negative': negative}
ftable = pd.DataFrame(data = d)

ftable.to_csv('ftable.csv')
print(positive_instance)
print(negative_instance)


In [ ]:
import pandas as pd



test = 'i am really happy birthday okay bob'

#split all the words in my text
test_words = test.str.split()



In [ ]:
import pandas as pd
import numpy as np

ftable = pd.read_csv('ftable.csv')
ftable = ftable[ftable['word'] != 'sad']
ftable = ftable[ftable['word'] != 'sad,']
ftable = ftable[ftable['word'] != ':(']
ftable = ftable[ftable['word'] != 'fun']
ftable = ftable[ftable['word'] != 'happy,']
ftable = ftable[ftable['word'] != 'happy']
ftable = ftable.drop_duplicates(subset = 'word')

test = 'i dont know what to do anymore'
positive_instance = 24070.0
negative_instance = 23930.0

#split all the words in my text
test_words = test.split()

prob_positive = float(positive_instance/(positive_instance+negative_instance))
prob_negative = 1 - prob_positive

pos_word = 1.0*prob_positive
neg_word = 1.0*prob_negative
for i in range(len(test_words)):
    word = test_words[i]
    #print(word)
    index_val = ftable.index[ftable['word'] == word]
    if (len(index_val) > 0):
        #print(index_val[0])
        pos_val = ftable['positive'].iloc[index_val[0]]
        neg_val = ftable['negative'].iloc[index_val[0]]
        pos_word = pos_word * pos_val/positive_instance
        neg_word = neg_word * neg_val/negative_instance
        
if pos_word > neg_word:
    print("The sentence was POSITIVE, with a probability of")
    print(pos_word/(pos_word+neg_word))
else:
    print("The sentence was NEGATIVE, with a probability of")
    print(neg_word/(pos_word+neg_word))

#print(pos_word)
#print(neg_word)
